# Метрики качества классификации

## 1. Данные

In [1]:
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
X = pd.read_pickle('data/shelter/res/xtrain.pkl')
y = pd.read_csv('data/shelter/train.csv')['OutcomeType']

для учебных примеров оставим только 2 класса, отличающихся значительно и по смыслу, и по размеру

In [4]:
y.value_counts()

Adoption           10769
Transfer            9422
Return_to_owner     4786
Euthanasia          1555
Died                 197
Name: OutcomeType, dtype: int64

In [5]:
is_first_2_class = (y=='Adoption')|(y=='Euthanasia')

In [6]:
X = X[is_first_2_class]
X.index = range(len(X))
y = y[is_first_2_class]
y.index = range(len(y))

y теперь можно перевести в множество {0,1}

In [7]:
y = (y=='Adoption').astype(int)

для оценки качества разобьём выборки на 2 части, 3:1

In [8]:
Xtr, Xval, ytr, yval = train_test_split(X, y, test_size=0.25)

-----

## 2. Обучим классификатор

In [9]:
from sklearn.tree import DecisionTreeClassifier

In [10]:
clf = DecisionTreeClassifier(max_depth=5)

In [11]:
clf.fit(Xtr, ytr)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [12]:
yval_pred = clf.predict(Xval)
yval_pred_proba = clf.predict_proba(Xval)

-----

## 3. Написание метрик с нуля

In [13]:
from sklearn import metrics

**hint:** в сами библиотеки можно заходить и смотрить код. Особенно это удобно в [PyCharm](https://www.jetbrains.com/pycharm/)

-----

Основа многих метрик - матрица ошибок (confusion matrix)

**Задание.** Написать функцию для расчёта матрицы ошибок при заданных *y, y_pred* и сравнить со значениями из sklearn.metrics

In [14]:
def get_confusion_matrix(y, y_pred, print_matrix=False):
    # приводим это в pandas.Series и избавляемся от индекса (иначе по нему будет идти сопоставление)
    y = pd.Series(list(y))
    y_pred = pd.Series(list(y_pred))
    
    # рассчитываем значения
    TP = sum((y==1)&(y_pred==1))
    FP = sum((y==0)&(y_pred==1))
    TN = sum((y==0)&(y_pred==0))
    FN = sum((y==1)&(y_pred==0))
    
    # выводим матрицу
    if print_matrix:
        print("{:5} {:5} {:5}".format("", "y=1", "y=0"))
        print("a=1 {:5} {:5}".format(TP, FP))
        print("a=0 {:5} {:5}".format(FN, TN))
    
    return {"TP":TP, "FP":FP, "TN":TN, "FN":FN}

In [15]:
get_confusion_matrix(yval, yval_pred, True)

      y=1   y=0  
a=1  2664   141
a=0    29   247


{'FN': 29, 'FP': 141, 'TN': 247, 'TP': 2664}

In [16]:
metrics.confusion_matrix(yval, yval_pred)

array([[ 247,  141],
       [  29, 2664]])

-----

**Задание.** Написать функцию для расчёта accuracy (доли верных ответов) и сравнить со значением из sklearn.metrics

$accuracy = \frac {TP+TN}{TP+TN+FP+FN}$

In [17]:
def accuracy(y, y_pred):
    return sum(y==y_pred)/len(y)

In [18]:
accuracy(yval, yval_pred)

0.94482310938007141

In [19]:
metrics.accuracy_score(yval, yval_pred)

0.94482310938007141

-----

**Задание.** Написать функцию для расчёта precision (точности) и сравнить со значением из sklearn.metrics

$precision = \frac {TP}{TP+FP}$

In [20]:
def precision(y, y_pred):
    return sum((y==1) & (y_pred==1))/sum(y_pred==1)

In [21]:
precision(yval, yval_pred)

0.94973262032085559

In [22]:
metrics.precision_score(yval, yval_pred)

0.94973262032085559

-----

**Задание.** Написать функцию для расчёта recall (полноты) и сравнить со значением из sklearn.metrics

$recall = \frac {TP}{TP+FN}$

In [23]:
def recall(y, y_pred):
    return sum((y==1) & (y_pred==1))/sum(y==1)

In [24]:
recall(yval, yval_pred)

0.98923134051243966

In [25]:
metrics.recall_score(yval, yval_pred)

0.98923134051243966

-----

**Задание.** Написать функцию для расчёта f1 score и сравнить со значением из sklearn.metrics

$f1 = 2*\frac {precision*recall}{precision+recall}$

In [26]:
def f1_score(y, y_pred):
    p = precision(y, y_pred)
    r = recall(y, y_pred)
    return 2*p*r/(p+r)

In [27]:
f1_score(yval, yval_pred)

0.96907966533284817

In [28]:
metrics.f1_score(yval, yval_pred)

0.96907966533284817

-----

**Задание.** Найти значения AUC-ROC, AUR-PRC, logloss с помощью sklearn

In [29]:
metrics.roc_auc_score(yval, yval_pred_proba[:,1])

0.92375134464686992

In [30]:
metrics.average_precision_score(yval, yval_pred_proba[:,1])

0.83473127633284283

In [31]:
metrics.log_loss(yval, yval_pred_proba[:,1])

0.26839362060015515